In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import unittest
import tqdm.notebook as tq
import solitaire_env
import sys
import seaborn as sns
import pickle as pkl
import gp
sys.setrecursionlimit(10**6)

In [3]:
with open("games/run_hardgame_2020-05-11 01:18:55.416668.pkl","rb") as file:
    run_data = pkl.load(file)

In [4]:
actions_matrix = run_data['actions_matrix']

In [5]:
iterations_used_per_game = run_data['iterations_used_per_game']
it = pd.DataFrame(data = {"iterations":iterations_used_per_game})['iterations'].value_counts()

In [6]:
max_action_take = max(it.index)
max_action_take 

93

In [7]:
w = 0
for game,actions,end_state,start_state,game_states,reason in actions_matrix:
    if len(actions) >60 and reason !='won':
        gno = game
        break
    if reason == 'won':
        w+=1
        #break
        
w

12

In [8]:
gno,reason,len(actions)

(634, 'No action could be taken', 62)

In [9]:
class replay_game:
    
    def __init__(self,gno=0):
        
        self.temp_en = solitaire_env.env()
        self.action_number = 0
        action_type = {0:'tableau_to_foundation_reveal',\
               1:'to_foundation_stack',2:'tableau_to_tableau_reveal',\
               3:'pile_to_tableau',\
                4:'foundation_to_tableau',\
               5:'tableau_to_tableau_not_reveal'\
              }
    
        for game_no,action_seq,end_state,start_state,state_seq,how_ends in actions_matrix:
            if game_no == gno:
                break;
        print("Reason game ended...   ",how_ends)
        print("Length of action sequence is.. ",len(action_seq))
        action_replay = "".join([str(x) for x in action_seq])
        
        self.action_replay = [int(x) for x in action_replay]
        
        self.temp_en.state = start_state
        self.start_state = start_state
        
        self.end_state = end_state
        
        
        self.temp_en.state = self.temp_en.copy_state()
        
        self.state_seq = state_seq
        
        self.action_seq = [action_type[int(x)] for x in action_replay]
        
        print("starting state.. \n")
        
        print("***********************************pile*****************************************")
        self.temp_en.get_pile()
        print("\n\n\n\n\n\n\n\n")
        print("*****************************foundation*****************************************")
        
        self.temp_en.get_foundation()
        print("\n\n\n\n\n\n\n\n")
        print("******************************tableau*******************************************")
        
        self.temp_en.get_tableau()
        print("\n\n\n\n\n\n\n\n")
        
    def next_state(self):
        
        print("action number..  ", self.action_number)
        
        print("action taken {} : description {} ".format(self.action_replay[self.action_number],self.action_seq[self.action_number]))
        self.action_number +=1
        self.temp_en = solitaire_env.env()
        
        self.temp_en.state = self.state_seq[self.action_number]
        
        self.temp_en.state = self.temp_en.copy_state()
        
        print("starting state.. \n")
        
        print("***********************************pile*****************************************")
        self.temp_en.get_pile()
        print("\n\n\n\n\n\n\n\n")
        
        
        print("*****************************foundation*****************************************")
        
        self.temp_en.get_foundation()
        print("\n\n\n\n\n\n\n\n")
            
            
        print("******************************tableau*******************************************")
        
        self.temp_en.get_tableau()
        print("\n\n\n\n\n\n\n\n")
        
        
        
        
    def goto_state(self,n):
        self.action_number = n

In [10]:
game_replay = replay_game(gno = 634)

Reason game ended...    No action could be taken
Length of action sequence is..  62
starting state.. 

***********************************pile*****************************************
position    0 suit = diamond    color = red        number =     1 speciality = ACE        face = up   
position    1 suit = spade      color = black      number =     2 speciality = None       face = up   
position    2 suit = club       color = black      number =    11 speciality = JACK       face = up   
position    3 suit = heart      color = red        number =     1 speciality = ACE        face = up   
position    4 suit = diamond    color = red        number =     5 speciality = None       face = up   
position    5 suit = heart      color = red        number =    11 speciality = JACK       face = up   
position    6 suit = heart      color = red        number =     8 speciality = None       face = up   
position    7 suit = heart      color = red        number =     4 speciality = None       face 

In [12]:
game_replay.goto_state(61)
game_replay.next_state()
#foundation progression

action number..   61
action taken 1 : description to_foundation_stack 
starting state.. 

***********************************pile*****************************************
position    0 suit = heart      color = red        number =    12 speciality = QUEEN      face = up   
position    1 suit = spade      color = black      number =    13 speciality = KING       face = up   
position    2 suit = heart      color = red        number =    10 speciality = None       face = up   









*****************************foundation*****************************************
printing foundation number 1 ********************
position    0 suit = club       color = black      number =     1 speciality = ACE        face = up   
position    1 suit = club       color = black      number =     2 speciality = None       face = up   
position    2 suit = club       color = black      number =     3 speciality = None       face = up   
position    3 suit = club       color = black      number =     4 speci

In [145]:
game_replay.next_state()

action number..   61
action taken 1 : description to_foundation_stack 
starting state.. 

***********************************pile*****************************************
position    0 suit = heart      color = red        number =    12 speciality = QUEEN      face = up   
position    1 suit = spade      color = black      number =    13 speciality = KING       face = up   
position    2 suit = heart      color = red        number =    10 speciality = None       face = up   









*****************************foundation*****************************************
printing foundation number 1 ********************
position    0 suit = club       color = black      number =     1 speciality = ACE        face = up   
position    1 suit = club       color = black      number =     2 speciality = None       face = up   
position    2 suit = club       color = black      number =     3 speciality = None       face = up   
position    3 suit = club       color = black      number =     4 speci

In [ ]:
# Tableau to foundation reveal #gamestate

In [13]:
temp_en = solitaire_env.env()

temp_en.state = game_replay.temp_en.state

In [19]:
temp_en.get_pile()

position    0 suit = heart      color = red        number =    12 speciality = QUEEN      face = up   
position    1 suit = spade      color = black      number =    13 speciality = KING       face = up   
position    2 suit = heart      color = red        number =    10 speciality = None       face = up   


In [20]:
temp_en.get_foundation()
temp_en.get_tableau()

printing foundation number 1 ********************
position    0 suit = club       color = black      number =     1 speciality = ACE        face = up   
position    1 suit = club       color = black      number =     2 speciality = None       face = up   
position    2 suit = club       color = black      number =     3 speciality = None       face = up   
position    3 suit = club       color = black      number =     4 speciality = None       face = up   
position    4 suit = club       color = black      number =     5 speciality = None       face = up   
position    5 suit = club       color = black      number =     6 speciality = None       face = up   
position    6 suit = club       color = black      number =     7 speciality = None       face = up   
position    7 suit = club       color = black      number =     8 speciality = None       face = up   
position    8 suit = club       color = black      number =     9 speciality = None       face = up   
position    9 suit = cl

In [17]:
w,t = temp_en.step(4,debug=True,fp_flag=True)

Debug is True


In [18]:
w,t

(False, False)